### 1. DCP Housing Data

In [2]:
import pandas as pd
import numpy as np
df=pd.read_csv("/Users/yujinlee/Google Drive/NYU/Spring 2023/ML_Cities/Project/HousingDB_post2010_completed_jobs.csv", low_memory=False)


In [3]:
df['Ownership'].head()

0    Private For-Profit: Corporation
1     Private For-Profit: Individual
2    Private For-Profit: Partnership
3    Private For-Profit: Corporation
4    Private For-Profit: Corporation
Name: Ownership, dtype: object

In [5]:
df.shape

(54505, 69)

In [6]:
df.head()

,Job_Number,Job_Type,ResidFlag,NonresFlag,Job_Status,CompltYear,PermitYear,ClassAInit,ClassAProp,ClassANet,...,FireBattln,FireDivsn,PolicePcnt,PL_FIRM07,PL_PFIRM15,Latitude,Longitude,GeomSource,DCPEdited,Version
0,102138820,Alteration,Residential,,5. Completed Construction,2012,2008,6,19,13,...,11,3,24,,,40.796735,-73.962554,Lat/Lon geosupport,otherb_init/classa_init/classa_prop/hotel_prop...,22Q2
1,102151083,Alteration,Residential,,5. Completed Construction,2011,2002,7,9,2,...,4,1,9,,,40.722607,-73.982247,Lat/Lon geosupport,,22Q2
2,102284172,Alteration,Residential,Non-Residential,5. Completed Construction,2016,2001,1,1,0,...,6,1,9,,,40.726393,-73.992359,Lat/Lon geosupport,classa_init,22Q2
3,102284804,Alteration,Residential,,5. Completed Construction,2022,2001,4,4,0,...,6,1,6,,,40.728743,-74.000837,Lat/Lon geosupport,classa_init/otherb_init/hotel_init/otherb_prop...,22Q2
4,102292010,Alteration,Residential,Non-Residential,5. Completed Construction,2022,2001,0,2,2,...,1,1,5,,,40.718334,-74.001400,Lat/Lon geosupport,otherb_init/otherb_prop/hotel_init/classa_init...,22Q2


In [7]:
df['BBL'].isnull().sum()

0

In [8]:
df['ResidFlag'].unique()

array(['Residential'], dtype=object)

In [9]:
df = df[['Job_Type','CompltYear','ClassANet','BBL','Enlargemnt']]

In [10]:
df.head()

,Job_Type,CompltYear,ClassANet,BBL,Enlargemnt
0,Alteration,2012,13,1018390020,
1,Alteration,2011,2,1003860063,
2,Alteration,2016,0,1005300132,
3,Alteration,2022,0,1005260062,
4,Alteration,2022,2,1001960027,


In [11]:
# If demolition ocurred after alteration, that alteration doesn't count. Drop these alterations.

#1) get the most recent demolition year 
df_bbl_demol = df[df['Job_Type'] == 'Demolition'].groupby(["BBL"])['CompltYear'].max().reset_index()
df_bbl_demol = df_bbl_demol.rename(columns={'CompltYear':'demol_year'})
df_bbl_demol.head(3)

,BBL,demol_year
0,1000010010,2020
1,1000390041,2016
2,1000520016,2019


In [12]:
df_merged = df.merge(df_bbl_demol, on=['BBL'], how='left')
df_merged.head(5)

,Job_Type,CompltYear,ClassANet,BBL,Enlargemnt,demol_year
0,Alteration,2012,13,1018390020,,NaN
1,Alteration,2011,2,1003860063,,NaN
2,Alteration,2016,0,1005300132,,NaN
3,Alteration,2022,0,1005260062,,NaN
4,Alteration,2022,2,1001960027,,NaN


In [13]:
#2) Only keep those happened after the most recent demolition (delete everything that happened before demolition)

df2 = df_merged[(df_merged['CompltYear'] > df_merged['demol_year']) 
    | df_merged['demol_year'].isnull() | (df_merged['CompltYear'] == '')]
df2 = df2.drop('demol_year', axis=1)

In [14]:
# Drop if Complete year > 2019

df2 = df2[df2.CompltYear < 2019]
df2

,Job_Type,CompltYear,ClassANet,BBL,Enlargemnt
0,Alteration,2012,13,1018390020,
1,Alteration,2011,2,1003860063,
2,Alteration,2016,0,1005300132,
6,Alteration,2010,2,1007937504,
7,New Building,2010,51,1004207501,
...,...,...,...,...,...
52991,New Building,2018,1,5010860098,
53001,Alteration,2018,0,5062280071,
53034,New Building,2018,2,5000420061,
53185,Alteration,2018,0,5030400212,


In [15]:
# Keep Alterations only

df2 = df2[df2.Job_Type == "Alteration"]
df2

,Job_Type,CompltYear,ClassANet,BBL,Enlargemnt
0,Alteration,2012,13,1018390020,
1,Alteration,2011,2,1003860063,
2,Alteration,2016,0,1005300132,
6,Alteration,2010,2,1007937504,
9,Alteration,2012,-114,1013050033,
...,...,...,...,...,...
52970,Alteration,2018,0,5036660017,
52980,Alteration,2018,1,5033620159,Horizontal and Vertical
53001,Alteration,2018,0,5062280071,
53185,Alteration,2018,0,5030400212,


In [16]:
df2['BBL'].unique().shape

(18351,)

In [17]:
# Make BBL the first column

col_list = df2.columns.tolist()
col_list.remove("BBL")
col_list = ["BBL"] + col_list
df2 = df2[col_list]

In [18]:
df2['has_enlarge'] = (df2['Enlargemnt'] != ' ').astype(int)
df2

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_76010/567850106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['has_enlarge'] = (df2['Enlargemnt'] != ' ').astype(int)


,BBL,Job_Type,CompltYear,ClassANet,Enlargemnt,has_enlarge
0,1018390020,Alteration,2012,13,,0
1,1003860063,Alteration,2011,2,,0
2,1005300132,Alteration,2016,0,,0
6,1007937504,Alteration,2010,2,,0
9,1013050033,Alteration,2012,-114,,0
...,...,...,...,...,...,...
52970,5036660017,Alteration,2018,0,,0
52980,5033620159,Alteration,2018,1,Horizontal and Vertical,1
53001,5062280071,Alteration,2018,0,,0
53185,5030400212,Alteration,2018,0,,0


In [23]:
# create a new col
# if complete year >= 2016 (altered in the last 5 years) , altered_lessthanfive
# if complete year < 2016 (altered more than 5 years ago), altered_morethanfive

df_recent_alter = df2.groupby(['BBL'])['CompltYear'].max().reset_index()
df_recent_alter

,BBL,CompltYear
0,1000057501,2015
1,1000080039,2014
2,1000110012,2010
3,1000130027,2011
4,1000150022,2015
...,...,...
18346,5080400023,2016
18347,5080460059,2016
18348,5080470090,2011
18349,5080480012,2015


In [24]:
df_recent_alter['altered_lessthan5yrs'] = np.where(df_recent_alter['CompltYear'] >= 2016, 1,0)
df_recent_alter['altered_morethan5yrs'] = np.where(df_recent_alter['CompltYear'] < 2016, 1,0)
df_recent_alter

,BBL,CompltYear,altered_lessthan5yrs,altered_morethan5yrs
0,1000057501,2015,0,1
1,1000080039,2014,0,1
2,1000110012,2010,0,1
3,1000130027,2011,0,1
4,1000150022,2015,0,1
...,...,...,...,...
18346,5080400023,2016,1,0
18347,5080460059,2016,1,0
18348,5080470090,2011,0,1
18349,5080480012,2015,0,1


In [29]:
df_recent_alter = df_recent_alter[['BBL','altered_lessthan5yrs','altered_morethan5yrs']]

In [26]:
# Count of Alteration - base df for merging

df_base = df2[df2['Job_Type']=='Alteration'].groupby('BBL')['Job_Type'].count().reset_index()
df_base.head()

,BBL,Job_Type
0,1000057501,1
1,1000080039,1
2,1000110012,1
3,1000130027,1
4,1000150022,1


In [30]:
# Merge with Alteration Year Columns, Average ClassANet and Enlargement Count datasets
print(df_base.shape)
df_merged = (df_base.merge(df_recent_alter, on = 'BBL')
                     .merge(df2.groupby(['BBL'])['ClassANet'].mean().reset_index(), on='BBL')
                     .merge(df2.groupby(['BBL'])['has_enlarge'].sum().reset_index(), on='BBL' )
             )
print(df_merged.shape)

(18351, 2)
(18351, 6)


In [31]:
df_merged

,BBL,Job_Type,altered_lessthan5yrs,altered_morethan5yrs,ClassANet,has_enlarge
0,1000057501,1,0,1,0.0,0
1,1000080039,1,0,1,-21.0,0
2,1000110012,1,0,1,1.0,0
3,1000130027,1,0,1,0.0,0
4,1000150022,1,0,1,0.0,0
...,...,...,...,...,...,...
18346,5080400023,1,1,0,0.0,0
18347,5080460059,1,1,0,0.0,1
18348,5080470090,1,0,1,0.0,0
18349,5080480012,1,0,1,0.0,0


In [32]:
df_merged[df_merged['Job_Type'] > 1]

,BBL,Job_Type,altered_lessthan5yrs,altered_morethan5yrs,ClassANet,has_enlarge
5,1000157501,2,0,1,0.0,0
7,1000160100,2,1,0,0.0,0
10,1000167511,2,1,0,0.5,0
22,1000417504,2,1,0,388.0,0
64,1001320027,2,0,1,0.0,0
...,...,...,...,...,...,...
17993,5053860076,2,0,1,0.0,0
18097,5062320007,2,0,1,0.0,0
18106,5062400060,2,0,1,-0.5,0
18229,5068490048,2,0,1,0.5,1


In [33]:
df_merged[df_merged['has_enlarge'] > 1]

,BBL,Job_Type,altered_lessthan5yrs,altered_morethan5yrs,ClassANet,has_enlarge
17154,5008280205,2,1,0,0.0,2


In [34]:
# rename colums

df_merged= df_merged.rename(columns={'Job_Type': 'Aleration_Count', 'ClassANet': 'ClassANet_Avg', 'has_enlarge':'Enlargement_Count'})
df_merged

,BBL,Aleration_Count,altered_lessthan5yrs,altered_morethan5yrs,ClassANet_Avg,Enlargement_Count
0,1000057501,1,0,1,0.0,0
1,1000080039,1,0,1,-21.0,0
2,1000110012,1,0,1,1.0,0
3,1000130027,1,0,1,0.0,0
4,1000150022,1,0,1,0.0,0
...,...,...,...,...,...,...
18346,5080400023,1,1,0,0.0,0
18347,5080460059,1,1,0,0.0,1
18348,5080470090,1,0,1,0.0,0
18349,5080480012,1,0,1,0.0,0


In [35]:
# save to csv - ready to merge

df_merged.to_csv('/Users/yujinlee/Google Drive/NYU/Spring 2023/ML_Cities/Project/HousingDB_post2010_completed_jobs_cleaned.csv')

## 2. Eviction

In [23]:
df_evic=pd.read_csv("/Users/yujinlee/Google Drive/NYU/Spring 2023/ML_Cities/Project/Evictions_2019.csv", low_memory=False)


In [24]:
df_evic.head()

,Court Index Number,Docket Number,Eviction Address,Eviction Apartment Number,Executed Date,Marshal First Name,Marshal Last Name,Residential/Commercial,BOROUGH,Eviction Postcode,Ejectment,Eviction/Legal Possession,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,71510/18,20934,97-34 ALLENDALE STREET,2A,01/02/2019,Edward,Guida,Residential,QUEENS,11435,Not an Ejectment,Possession,40.696205,-73.808137,12.0,28.0,206.0,4214281.0,4.100130e+09,Jamaica
1,B22065/18,91616,3018 HOLLAND AVENUE,B1,01/02/2019,Ileana,Rivera,Residential,BRONX,10467,Not an Ejectment,Possession,40.870045,-73.865244,12.0,15.0,338.0,2055605.0,2.045700e+09,Bronxdale
2,71510/18,20349,97-34 ALLENDALE STREET,2A,01/02/2019,Edward,Guida,Residential,QUEENS,11435,Not an Ejectment,Possession,40.696205,-73.808137,12.0,28.0,206.0,4214281.0,4.100130e+09,Jamaica
3,50809/18,76903,142 BRIGHTON AVE.,NaN,01/02/2019,Kenneth,Giachetta,Residential,STATEN ISLAND,10301,Not an Ejectment,Possession,40.635486,-74.090517,1.0,49.0,75.0,5002646.0,5.001120e+09,New Brighton-Silver Lake
4,75821/18,9789,114 SUFFOLK STREET - APT 3B (3RD FL),3B,01/02/2019,Robert,Renzulli,Residential,MANHATTAN,10002,Not an Ejectment,Possession,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_evic.shape

(17036, 20)

In [26]:
df_evic['BBL'].isnull().sum()

1200

In [27]:
df_evic = df_evic.dropna(subset=['BBL'])

In [28]:
df_evic['BBL'].unique().shape

(9911,)

In [29]:
df_evic['Residential/Commercial'].unique()

array(['Residential'], dtype=object)

In [30]:
df_evic['Ejectment'].unique()

array(['Not an Ejectment', 'Ejectment'], dtype=object)

In [31]:
df_evic['Ejectment'].unique()

array(['Not an Ejectment', 'Ejectment'], dtype=object)

In [32]:
df_evic['Ejectment'].isnull().sum()

0

In [33]:
df_evic['Eviction/Legal Possession'].unique()

array(['Possession', 'Eviction'], dtype=object)

In [34]:
df_evic['Eviction/Legal Possession'].isnull().sum()

0

In [35]:
df_possession = df_evic[df_evic['Eviction/Legal Possession']=='Possession'].groupby('BBL')['Eviction/Legal Possession'].count().reset_index()
df_possession

,BBL,Eviction/Legal Possession
0,1.000158e+09,2
1,1.000160e+09,5
2,1.000168e+09,1
3,1.000168e+09,1
4,1.000168e+09,1
...,...,...
9816,5.078670e+09,1
9817,5.078800e+09,2
9818,5.078910e+09,1
9819,5.080260e+09,2


In [36]:
df_eviction = df_evic[df_evic['Eviction/Legal Possession']=='Eviction'].groupby('BBL')['Eviction/Legal Possession'].count().reset_index()
df_eviction

,BBL,Eviction/Legal Possession
0,1.001110e+09,4
1,1.002550e+09,1
2,1.002560e+09,1
3,1.002580e+09,1
4,1.002600e+09,2
...,...,...
162,5.000710e+09,1
163,5.001960e+09,7
164,5.005450e+09,2
165,5.012450e+09,2


In [37]:
df_evic['Ejectment_binary'] = (df_evic['Ejectment'] == 'Ejectment').astype(int)
df_evic['Ejectment_binary']

0        0
1        0
2        0
3        0
5        0
        ..
17030    0
17032    0
17033    0
17034    0
17035    0
Name: Ejectment_binary, Length: 15836, dtype: int64

In [38]:
df_evic_base = df_evic.groupby('BBL')['Ejectment_binary'].sum().reset_index()
df_evic_base

,BBL,Ejectment_binary
0,1.000158e+09,0
1,1.000160e+09,0
2,1.000168e+09,0
3,1.000168e+09,0
4,1.000168e+09,0
...,...,...
9906,5.078670e+09,0
9907,5.078800e+09,0
9908,5.078910e+09,0
9909,5.080260e+09,0


In [39]:
print(df_evic_base.shape)
df_evic_merged = (df_evic_base.merge(df_evic[df_evic['Eviction/Legal Possession']=='Possession'].groupby('BBL')['Eviction/Legal Possession'].count().reset_index(), on='BBL', how = 'left')
                     .merge(df_evic[df_evic['Eviction/Legal Possession']=='Eviction'].groupby('BBL')['Eviction/Legal Possession'].count().reset_index(), on='BBL', how = 'left'))
print(df_evic_merged.shape)

(9911, 2)
(9911, 4)


In [40]:
df_evic_merged

,BBL,Ejectment_binary,Eviction/Legal Possession_x,Eviction/Legal Possession_y
0,1.000158e+09,0,2.0,NaN
1,1.000160e+09,0,5.0,NaN
2,1.000168e+09,0,1.0,NaN
3,1.000168e+09,0,1.0,NaN
4,1.000168e+09,0,1.0,NaN
...,...,...,...,...
9906,5.078670e+09,0,1.0,NaN
9907,5.078800e+09,0,2.0,NaN
9908,5.078910e+09,0,1.0,NaN
9909,5.080260e+09,0,2.0,NaN


In [41]:
df_evic_merged = df_evic_merged.fillna(0)

In [42]:
df_evic_merged = df_evic_merged.rename(columns={'Ejectment_binary':'ejectment_count', 'Eviction/Legal Possession_x': 'possession_count', 'Eviction/Legal Possession_y':"eviction_count"})
df_evic_merged

,BBL,ejectment_count,possession_count,eviction_count
0,1.000158e+09,0,2.0,0.0
1,1.000160e+09,0,5.0,0.0
2,1.000168e+09,0,1.0,0.0
3,1.000168e+09,0,1.0,0.0
4,1.000168e+09,0,1.0,0.0
...,...,...,...,...
9906,5.078670e+09,0,1.0,0.0
9907,5.078800e+09,0,2.0,0.0
9908,5.078910e+09,0,1.0,0.0
9909,5.080260e+09,0,2.0,0.0


In [43]:
df_evic_merged.to_csv('/Users/yujinlee/Google Drive/NYU/Spring 2023/ML_Cities/Project/Evictions_2019_cleaned.csv')